In [1]:
import numpy as np
import os, sys
sys.path.append('../')
from src.datasets.soccernet_generic import soccernet_dataset_generic
from src.utils.helper import samples_by_language
import src.utils.training_helper as training_helper
import torch
from torch.utils.data import Dataset, DataLoader

/home/oarongve/.local/share/virtualenvs/project-daredevil-8eBKzQn6/lib/python3.6/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
root_dir = "/work/oarongve/data/sound_dataset/SoccerNet-code/data/"
train_list = "/work/oarongve/data/sound_dataset/SoccerNet-code/data/listgame_Train_300.npy"
valid_list = "/work/oarongve/data/sound_dataset/SoccerNet-code/data/listgame_Valid_100.npy"
test_list = "/work/oarongve/data/sound_dataset/SoccerNet-code/data/listgame_Test_100.npy"


In [3]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
%%bash
nvidia-smi

Mon Dec 14 00:28:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.165.02   Driver Version: 418.165.02   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:BE:00.0 Off |                    0 |
| N/A   34C    P0    49W / 350W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [5]:
langpath_train = '/work/oarongve/project-daredevil/project-daredevil/language-annotations/annotations/train_lang_dict.json'
langpath_valid = '/work/oarongve/project-daredevil/project-daredevil/language-annotations/annotations/valid_lang_dict.json'
langpath_test = '/work/oarongve/project-daredevil/project-daredevil/language-annotations/annotations/test_lang_dict.json'

samples_train_all = samples_by_language(langpath_train,train_list,'all')
samples_valid_all = samples_by_language(langpath_valid,valid_list,'all')
samples_test_all = samples_by_language(langpath_test,test_list,'all')


In [6]:
train_set_all = soccernet_dataset_generic(npy_file=train_list,root_dir=root_dir,lang='all',lang_dict=langpath_train)
valid_set_all = soccernet_dataset_generic(npy_file=valid_list,root_dir=root_dir,lang='all',lang_dict=langpath_valid)
test_set_all = soccernet_dataset_generic(npy_file=valid_list,root_dir=root_dir,lang='all',lang_dict=langpath_valid)


In [7]:
params = {'batch_size': 32,
         'shuffle': True,
         'num_workers':4}

# window size

w = 8

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
train_set_all.load_waves()
valid_set_all.load_waves()
test_set_all.load_waves()

train_set_all.generate_mel_spectrograms(load_features=True)
valid_set_all.generate_mel_spectrograms(load_features=True)
test_set_all.generate_mel_spectrograms(load_features=True)

train_set_all.load_resnet_features()
valid_set_all.load_resnet_features()
test_set_all.load_resnet_features()

100%|██████████| 100/100 [00:01<00:00, 87.70it/s]


In [9]:
# window size
train_set_all.set_window_size(w)
valid_set_all.set_window_size(w)


In [10]:
trainloader_all = DataLoader(train_set_all,**params)
validloader_all = DataLoader(valid_set_all,**params)


In [11]:
train_set_all[0]['ms_spot'].shape

torch.Size([128, 320])

# Case 1: Trained on all, test and evaluate on all, english, other

In [13]:
window_positions = ['center','forward','back']

In [16]:
import torchvision
import torch.optim as optim
import torch.nn as nn
def generate_model():
    model = torchvision.models.resnet18(pretrained=True)
    model.conv1 = nn.Conv2d(1,64,kernel_size=(7,7),stride=(2,2),padding=(3,3),bias=False)
    model.fc = nn.Linear(512,4,bias=True)
    #resnet.features.conv0 = nn.Conv2d(1, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    #resnet.classifier = nn.Linear(in_features=densenet.classifier.in_features, out_features=3,bias=True)


    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    return model, criterion,optimizer

In [17]:
window_pos_accs = list()
for pos in window_positions:
    train_set_all.set_window_position(pos)
    valid_set_all.set_window_position(pos)
    
    trainloader = DataLoader(train_set_all,**params)
    validloader = DataLoader(valid_set_all,**params)
    
    model, criterion,optimizer = generate_model()
    # Scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.1)
    resnet,eng = training_helper.train_model_ws(model=model,
           epochs=25,
           trainloader=trainloader,
           validloader=validloader,
           criterion=criterion,
           optimizer=optimizer,
           scheduler=scheduler,
           device=device,
        tensorboard_name="window_size_25e_v5_"+str(pos))
    
    best_acc = training_helper.evaluate_model(resnet,validloader,device)
    window_pos_accs.append(best_acc)
    

new best acc
 Accuracy : 0.6066666841506958
new best acc
 Accuracy : 0.681538462638855
 Accuracy : 0.5917948484420776
 Accuracy : 0.6507692337036133
 Accuracy : 0.6441025733947754
 Accuracy : 0.4984615445137024
 Accuracy : 0.6487179398536682
 Accuracy : 0.6743589639663696
 Accuracy : 0.6179487109184265
new best acc
 Accuracy : 0.7128205299377441
new best acc
 Accuracy : 0.7133333086967468
new best acc
 Accuracy : 0.717435896396637
 Accuracy : 0.7158974409103394
new best acc
 Accuracy : 0.7189743518829346
 Accuracy : 0.7158974409103394
 Accuracy : 0.7041025757789612
 Accuracy : 0.7107692360877991
 Accuracy : 0.7184615135192871
 Accuracy : 0.7143589854240417
 Accuracy : 0.7123076915740967
 Accuracy : 0.7092307806015015
 Accuracy : 0.7138461470603943
 Accuracy : 0.7148718237876892
 Accuracy : 0.7097436189651489
 Accuracy : 0.708717942237854
Finished Training
Accuracy : 0.7189743518829346
new best acc
 Accuracy : 0.6066666841506958
new best acc
 Accuracy : 0.6143589615821838
 Accuracy : 0.

In [18]:
print(window_pos_accs)

[tensor(0.7190), tensor(0.6826), tensor(0.6544)]
